

Group nr:

Name 1 and CID: Fredrik Sandén (Sandenfr)

Name 2 and CID: Simon Holmgren (Simh)

In [1]:
import numpy as np
import copy
import random
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from mining_world import Environment
from IPython.display import Image


pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Introduction

This assignment dels with data structures, K-NN, tree-based classifiers and evaluation of the methods deployed on a fictional scenario.  

## Mining world 

<img src="imgs/poster.png" width="800"/>

## Scenario


Humanity has now reached a point where we need to extract and refine more Copium, a precious resource with great value. The only problem is that Copium can only be found on certain uninhabitable planets. This of course means that automated robots are sent instead.      

Copium is naturally very unstable and is only exists very temporary before it decays. There are very specific geological activities and circumstances needed for copium to form. The life cycle of Copium follows. First, a hot stream of liquid magma flows to the surface, creating a hotspot that that looks like a small crater. At the surface, if the conditions are correct, copium can form during the cool-down period. But as stated previously, Copium is unstable in its natural environment and decays to other materials shortly after. 

The formation of these deposits craters are very random, but the heat from them can easilly be detected with a satellite. But there is no way of knowing if the newly formed depoist contains copium from just a satellite, therefor there is a robot rover on the ground with sensors that can collect further measurements. The rover has many sensors that can measure the properties of the ground below it, but of course, Copium can not directly be detected with these types of sensors. This is where the machine learning approach will be used, to take all those measurements and try to classify if the deposit contains Copium or not.  


<img src="imgs/overView.png" width="500"/>


# Navigation - Tree search

This section will show how the naivigation is done. This is not a part of the assignment to understand, but will be used. 

##  Breadth first

The method used is a breadth first search algorithm, it is one of the simplest tree search algorithms and basically tries every option for a fixed number of steps and chooses the best one. The robot only have 4 actions, it can move at every time step in a chosen direction. 

## Actions 

<img src="imgs/actions.png" width="300"/>


In [2]:
class Node():
    def __init__(self, actor):
        self.actor = actor
        self.total_score = 0
    
    def update(self, action, inherited_score):
        score = self.actor.step(action)
        self.total_score = 1.05*inherited_score + score
        return self.total_score
    
    def get_score(self):
        return self.total_score

In [3]:
def breadth_first_search(actor, max_depth, action_space):
    node = Node(copy.deepcopy(actor)) 
    queue_keys = ['0'] # queue to keep track of nodes that has not yet been expanded.  
    visited = {queue_keys[0]: node} # saves visited nodes in order to not recalulate the entire path for each step. 
    
    max_score = -np.inf
    best_action = None

    while True:
        key = queue_keys.pop(0)
        if len(key) > max_depth: # stop at a set depth 
            break    
        node = visited[key]
        
        for action in action_space: # expand all children nodes
            child_node = copy.deepcopy(node)  # copy current node
            score = child_node.update(action=action, inherited_score=node.get_score()) # update node with action
            child_key = key + action # create child node key
            
            if score > max_score: # save best path 
                max_score = score
                best_action = child_key[1]
                
            visited[child_key] = child_node  # add child node to visited nodes.
            queue_keys.append(child_key)  # add child node queue of non expanded nodes. 
            
    return best_action


# Collect data

The first step is to collect data that will be used for both training and validation. The available types features can be seen with env.get_sensor_properties() and the actual measurements can be retrieved with env.get_sensor_readings(). It will return a dictionary with the same keys as in env.get_sensor_properties() containg a value for each feature. If the robot is not currently over a deposit, then it will return None. The true label can be extracted with env.get_ground_truth(), which will return a 1 if there is copium in the deposit and 0 if not. 
  

In [4]:
# Have True for first time running and then set to False. 
collect_data = False



In [5]:
env = Environment(map_type=1, fps=500, resolution=(1000, 1000))
env.exit()
sensor_properties = env.get_sensor_properties()
print('Sensor properties', sensor_properties)


Sensor properties ['ground_density', 'moist', 'reflectivity', 'silicon_rate', 'oxygen_rate', 'iron_rate', 'aluminium_rate', 'magnesium_rate', 'undetectable']


In [6]:
if collect_data == True:
    env = Environment(map_type=1, fps=500, resolution=(1000, 1000))
    sensor_properties = env.get_sensor_properties()

    # We can initilize the dictionary the following way.
    data = dict()
    data['copium'] = [] 
    for key in sensor_properties:
        data[key] = []

    for i in range(5000):
        action = breadth_first_search(actor=env.get_actor(), max_depth=3, action_space=env.get_action_space())
        env.step(action)
        # if we are over a deposit. 
        if env.get_sensor_readings() is not None:
            sensor_readings = env.get_sensor_readings()
            copium = env.get_ground_truth()

            for key in sensor_readings:
                data[key].append(sensor_readings[key])
            data['copium'].append(copium)

        env.render()

    env.exit()


# Exerscie 1: Data structure

## a) Pandas data frame 
In this assignment we will work pandas data frame for storing the collected data. First create a pandas data frame from the dictionary. The documentation for it can be found at https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html, only the data feild needs to be filled in with the created dictionary. Call this data frame df. 


In [7]:
# TODO: create pandas dataframe
if collect_data == True:
    df = pd.DataFrame(data)



## b) Save data
Collecting data for each time the notebook is opened is time consuming and the tests are not as reproducable if the data is different each time. Therefore we save the dateframe and we can load the data instead. After this you can set collect_data=False above. Use the function pandas function df.to_csv('name_of_file.csv', index=False)

The task is to save the data in a file named mydata.csv

In [8]:

# TODO: save data to a csv file named mydata.  
if collect_data == True:
    df.to_csv('mydata.csv', index=False)

## c) Load data
The task is to load the data from mydata.csv into a dataframe named df. Use the function df = pd.read_csv('name_of_file.csv')

In [9]:

# TODO: read data from mydata.csv into a dataframe called df.  
df = pd.read_csv('mydata.csv')

In [10]:
# From the data frame you can access all data for a key with for example:
print("All data for a feature \n", df["copium"])
print()

# You can access a single sample with:
print("Single sample from index \n", df.iloc[1])
print()

# You can access all freatures but one with:
all_features_without_copium = df.drop(columns='copium')
print("All features without copium \n", all_features_without_copium)

# To get an overview 
df.describe()

All data for a feature 
 0       0
1       0
2       0
3       0
4       0
       ..
1590    0
1591    1
1592    0
1593    0
1594    0
Name: copium, Length: 1595, dtype: int64

Single sample from index 
 copium            0.000000
ground_density    1.843045
moist             0.241389
reflectivity      0.569418
silicon_rate      0.120205
oxygen_rate       0.018468
iron_rate         0.653094
aluminium_rate    0.096327
magnesium_rate    0.065495
undetectable      0.046411
Name: 1, dtype: float64

All features without copium 
       ground_density     moist  reflectivity  silicon_rate  oxygen_rate  \
0           0.739599  0.107341      0.490689      0.020877     0.043239   
1           1.843045  0.241389      0.569418      0.120205     0.018468   
2           0.400000  0.131172      0.301149      0.047718     0.068963   
3           1.439975  0.188397      0.125550      0.139022     0.032345   
4           0.400000  0.224848      0.493306      0.181588     0.000502   
...              ... 

,copium,ground_density,moist,reflectivity,silicon_rate,oxygen_rate,iron_rate,aluminium_rate,magnesium_rate,undetectable
count,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000,1595.000000
mean,0.170533,1.413588,0.150130,0.300139,0.137604,0.041467,0.403441,0.259109,0.107806,0.050573
std,0.376218,0.642583,0.086313,0.176773,0.076981,0.027730,0.160909,0.127567,0.064500,0.029651
min,0.000000,0.400000,0.000147,0.001082,0.011549,0.000212,0.054140,0.020606,0.006451,0.005778
25%,0.000000,0.933412,0.078233,0.145155,0.076663,0.019467,0.274809,0.161031,0.056553,0.027935
50%,0.000000,1.297188,0.148299,0.301798,0.131738,0.039022,0.423402,0.255889,0.100420,0.046725
75%,0.000000,1.803201,0.226337,0.456903,0.182463,0.058084,0.526973,0.343514,0.146767,0.066417
max,1.000000,4.497465,0.299798,0.599848,0.470706,0.193886,0.793212,0.687683,0.369549,0.209317


## d) Part 1: Split data

Here we will devide the data into a training set and a test set. Good rule of thumb is to use 80% of the data in the training set and 20 % in the test set. The the two data sets should be randomly sampled (shuffle=True), we also set he seed with random_state so that the split is reproducable. This is done with train_test_split() from sklearn, https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html. The syntax looks like 

train, test = train_test_split(dataframe, test_size=ratio_test_set, shuffle=True, random_state=42)

Why is it important that the data is shuffled when it is split, what could happen otherwise?

Text answer:\
Since we want to train the modle for the entire data set. If we don't shuffle it we will only learn the modle on ex the first 80% of the data, missing the last 20%. So it's better to use the shuffled data instead. 



In [11]:
# TODO: devide the data into train and test set. 
train, test = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)


## e) Part 1: Analyse data balance

The occurance can be retrevied with .value_counts() from a pandas date frame. Here get the occurance of copium in the samples. Is the dataset balanced?

In [12]:
# TODO: Print number of samples with copium and the number of samples without copium.
dfc = df.value_counts("copium")
print("Sample with copium:", dfc[1])
print("Sample without copium:", dfc[0])



Sample with copium: 272
Sample without copium: 1323


Text answer:\
Since the samples with copium is not equal to the the amount of samples with copium the dataset is inbalanced. 

## e) Part 2: Balance data

We will throughout this assignment test each method with the original dataset and a balanced dataset. We show how balanceing can be done by downsampling below. Your task is to create an upsampled balanced dataset. You only need to use the upsampled data set for the rest of the other part 2) exerices. 

What could be the reason for chosing either upsampled or downsampeled metohd to balance the dataset?


Text answer:\
To get an optimal amount of datapoints. There may be issues if the amount of datapoints is to low or to high. 



In [13]:
# TODO balance data det. 
# Seperate the data into something that contains copium and one that doesn't,
# can for example be done with train[train["copium"]==0] etc.
train_zero = train[train["copium"]==0]
train_one = train[train["copium"]==1]

# downsample majority
train_zero_downsampled = resample(train_zero,
                               n_samples=train_one.shape[0])

train_balanced_downsampled = pd.concat([train_one, train_zero_downsampled])


# TODO: upsample minority 
train_one_upsampled = resample(train_one,
                               n_samples=train_zero.shape[0])

train_balanced_upsampled = pd.concat([train_zero, train_one_upsampled])


train_balanced_downsampled_count = train_balanced_downsampled.value_counts("copium")
train_balanced_upsampled_count = train_balanced_upsampled.value_counts("copium")
print("Upsampled data:",train_balanced_upsampled_count)
print("Downsampled data:", train_balanced_downsampled_count)


Upsampled data: copium
0    1055
1    1055
Name: count, dtype: int64
Downsampled data: copium
0    221
1    221
Name: count, dtype: int64


# Exercise 3: Performance evaluation

Here we will define a class that later will be used for evaluation the performance of the classification methods. It will be used to log the accuracy, precsision and recalll. More information about precision and recall can be found at https://en.wikipedia.org/wiki/Precision_and_recall. 

Expliain why the different metics are usefull and why is not always accuarcy enough?

Text answer:



In [14]:
class Classification_eval(object):
    def __init__(self):
        # counters 
        self.TP = 0 # Predicted positive, positive label
        self.FP = 0 # Predicted positive, negative label
        self.TN = 0 # Predicted negative, negative label
        self.FN = 0 # Predicted negative, positive label. 
    
    def update(self, pred, label):
        """
        pred - is the prediction will be either a 1 or 0. 
        label - is the correct answer, will be either a 1 or 0.
        """
        # TODO: add to one of the correct counter each time this function is called. 
        if pred and label: self.TP += 1
        if pred and not label: self.FP += 1
        if not pred and not label: self.TN += 1
        if not pred and label: self.FN += 1

    def accuracy(self): 
        # returns the accuracy 
        if (self.TP + self.TN) == 0:
            return 0
        # TODO: calculate the accuracy.
        accuracy = (self.TP + self.TN)/(self.TP + self.TN + self.FP + self.FN)
        return np.round(accuracy, 4)
    
    def precision(self): # percentage of the estimated positive that actually is positive
        if (self.TP + self.FP) == 0:
            return 0
        # TODO: calculate the precision.
        
        precision = self.TP/(self.TP + self.FP)
        return np.round(precision, 4)
    
    def recall(self): # percentage of correctly identified positive of the total positive
        if (self.TP + self.FN) == 0:
            return 0
        # TODO: calculate the recall.
        recall = (self.TP)/(self.TP + self.FN)
        return np.round(recall, 4)

In [15]:
# Test Classification_eval

test_eval = Classification_eval()
test_eval.update(1,1)
test_eval.update(1,1)
test_eval.update(1,1)
test_eval.update(0,1)
test_eval.update(1,0)
test_eval.update(1,0)
test_eval.update(0,0)
test_eval.update(0,0)
test_eval_ = np.allclose([test_eval.accuracy(), test_eval.precision(), test_eval.recall()], [0.625, 0.6, 0.75])
if test_eval_:
    print('Test successful')
else:
    print('Test failed')
assert test_eval_
   


Test successful


# Exercise 4: K- nearest neighbours

## a) Normalize

Here we will code our K-NN classifier, method 2.1 on page 21 in the book has the psudo code for K-NN. We will start with the data normalization, i.e. we will normalize the input data so that each feature has the same range in terms of max/min values. The min value can be found with data.min(), similarly for the max value. 

Why is it important that the data is normalized for the K-NN algorithm?

Text answer:\
To make sure you are searching in the values from smallest to biggest and avoid clumping them. 


In [16]:
class Normalize(object):
    def __init__(self):
        self.min = None
        self.max = None
    
    def normalize(self, data):
        # normalize the data and return it. 
        return (data-self.min)/(self.max-self.min)
    
    def update_normalization(self, data):
        # Save the min and max values for each feature. This funciton is only used for the training data.
        self.min = data.min()
        self.max = data.max()
        return self.normalize(data)

    

## b) K-NN
Lets make the k-Nearest Neighbors (k-NN) algorithm, fill in the TODO. 

The core idea behind k-NN is to make predictions based on the "neighborhood" of data points in a feature space. In this impementation we will use the $l^2$-norm for distance to determine the k nearest samples. The classification is done with a majoiry vote of the k nearest samples.

In [17]:

class KNN(object):
    def __init__(self, k):
        self.features = None # normalized features from training data 
        self.labels = None # the corresponding labels (if there is copium)
        self.normalize = Normalize() # class instance for normalization
        self.k = k # the k value in k-nn algorithm.
        
    def fit(self, features, labels):
        # This is where we save the training data. 
        # TODO: update the normalize filter and normalize the features (save to self.features)
        # and save the labels to self.labels. Hint see cell bellow for how the inputs looks like
        
        self.features = self.normalize.update_normalization(features)
        self.labels = labels

    def predict(self, sample):
        # For this part you are not allowed to use an existing K-NN package. 
        # This function takes one sample as an input and outputs a predicion 
        
        # TODO: 
        # First normalize the input sample. 
        # Find the k closest samples in self.features to the input sample
        # Return the prediction from the majority vote.  
        normalized_sample = self.normalize.normalize(sample)
        
        distance_list = [sum((self.features.iloc[i]-normalized_sample)**2) for i in range(self.features.shape[0])]

        k_index = np.argsort(distance_list)[:self.k]

        copium = [self.labels.iloc[i] for i in k_index]

        return np.bincount(copium).argmax()
            
        #return predicion


In [18]:
# Test the KNN against known values. The test only covers small sample and there are hidden tests which will 
# tested after submitting. 

test_train = pd.read_csv('test.csv')
test_test = pd.read_csv('test2.csv')

train_labels = test_train['copium']
train_features = test_train.drop(columns='copium')
test_labels = test_test['copium']
test_features = test_test.drop(columns='copium')

knn = KNN(k=5)
knn.fit(train_features, train_labels)
pred_list = []

for i in range(test_features.shape[0]):
    pred = knn.predict(test_features.iloc[i])
    pred_list.append(pred)
val = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
if np.allclose(pred_list, val):
    print('Test successful')
else:
    print('Test failed')
    

    


Test successful


## c) part 1: Evaluate the K-NN 
Evaluate the K-NN and choose a suitable k value. 

In [19]:
#train_labels = train['copium']
#train_features = train.drop(columns='copium')

train_labels = train_balanced_upsampled['copium']
train_features = train_balanced_upsampled.drop(columns='copium')

y = test['copium']
x = test.drop(columns='copium')

# TODO, try differenent values of k. 
knn = KNN(k=1)
knn.fit(train_features, train_labels)

log = Classification_eval()
for i in range(x.shape[0]):
    pred = knn.predict(x.iloc[i])
    log.update(pred, y.iloc[i])

print('Accuarcy', log.accuracy())
print('Precision', log.precision())
print('Recall', log.recall())


Accuarcy 0.8683
Precision 0.6216
Recall 0.451


Try some differnet values of k and just looking at these resutlts would the klassifier work well for all k? 

Answer:\
With the higher K values the accuracy is lower. This migh be because the higher K values may lead to generalisation issues in the model. However the precision value is higher and the recall value is lower which is better than the lower K values.

| k | Accuracy | Precision | Recall | 
| --- | --- | --- | --- |
| 1 | 0.8621 | 0.5897 | 0.415 |  
| 5 | 0.8871 | 0.8261 | 0.3725 |   
| 20 | 0.8527 | 1.0 | 0.0784 |   
| 50 | 0.8495 | 1.0  | 0.0588 |  


## c) part 2, balanced data. 
Now try with the upsampled balanced data, test for the same k values as in part 1. Are the results different and whitch one would identify more copium?  

Answer:\
There is a noticable improvement from the un-balanced data. However the precision and recall values are wors than the un-balanced data. This migh be fals posetive issue comping from the overrepresentation of posetive outcomes in the data set. 

| k | Accuracy | Precision | Recall | 
| --- | --- | --- | --- |
| 1 | 0.8683 | 0.6216 | 0.451 |  
| 5 | 0.8307 | 0.4805 | 0.7255 |   
| 20 | 0.8527  | 0.5323 | 0.6471 |   
| 50 | 0.8464 | 0.5156 | 0.6471|

## d) kNN with Sklearn
Typically one would not implement these algorithms by hand for each time you need them. Here we show how the kNN algorithm from sklearn can be used. Note, it does not normalize the features by default. The advantanges with the sklearn version is in computational cost as they utilize clever tree structures to minimize the search problem. 

A quick note: Even if ready packages exist, it is still very valuable to understand the principels behind the algorithms. For example when the learning doesn't work as well as you liked (happens more often then not...), you can reason to why and modify the problem or method into something that does work.  

In [20]:
# Set up data used for fitting the model
train_labels = train['copium']
train_features = train.drop(columns='copium')

# with the off the shelf model we need normalize the data before 
normalize = Normalize()
data_features = normalize.update_normalization(train_features)

# The test data
y = test['copium']
x = test.drop(columns='copium')

# Normalize the test data
x_norm = normalize.normalize(x)

# Set up an knn with sklearn 
knn_sklearn = KNeighborsClassifier(n_neighbors=5)
knn_sklearn.fit(data_features.values, train_labels.values)

# Set up our own for comparison 
knn = KNN(k=5)
knn.fit(train_features, train_labels)

# Test both for the 20 first values
for i in range(20):
    pred = knn.predict(x.iloc[i])
    pred_sklearn = knn_sklearn.predict([x_norm.iloc[i]])
    print('Prediciton from our model', pred, ' and from sklearn ',pred_sklearn)


Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  and from sklearn  [0]
Prediciton from our model 0  an

# Exercise 5: Tree based classifier

In this exercise we will implement a thee based classifier. The implementation will be a class representing a node and will recursivly create child node until the entire tree structure is built. The general step for learning a tree structure from learning data is as follows:

1. Initiate the the root node.
2. Call the learn function with the learning data. 

Inside the learn function

3. Check if all the data has the same classification, and check if the data set is smaller then min_node_size. If any of these are true, set the node as a leaf node and return. 
4. Find the spliting point in the data that returns the highest gini value, be careful to not use the label which we want to classify. 
5. Split the dataset, one is usually called head and the other tail. Save the splitting parameter and which value was used for the splitting. 
6. Check the prediction of the splited data sets (majority copium or not) and create two child nodes, one for each of the splited dataset. 
7. Call the the learn function of the child node with the corresponding split data and appdend the child node. After that return. 

The above will build the tree structure. To use the tree for prediction simply call the predict function of the root node with a data sample. In the predict function the following steps should happen.

1. Check if the current node is a lead node, if it is simply return the self.prediction of that node. 
2. If it is not a leaf node then check the value of the splitting parameter in the data sample and compare with the splitting value. 
3. Call the predict function of the correspoinding child node with the data sample and return the answer.

This will recurivly traverse the tree until a leaf node containing the prediction is reached. 

 


## 5 a) Find split point

First we define a function that can find the splitting point with the lowest gini value for a feature. For example, what is the best way to split the data with regards to the reflectivity feature. Page 34 in the book describes Gini index.  

The gini value can be described as:

If $\Gamma$ contains the set of all labeles, then $\Gamma(x < c)$ would be all labels that belong to the criteria $x < c$, where $x$ is a feature or parameter and $c$ is the splitting criteia (split_value).  


For example: 

If $x$ is ironRate and our splitting critera is $c = 0.4$, then $\Gamma(x < c)$ would be a set of all the label from samples with $\textrm{ironRate} < 0.4$. Remeber that our labels are either 0 or 1 in this assignment. 


Then we can define:

$r_1 = mean(\Gamma(x < c))$

$r_2 = mean(\Gamma(x \geq c))$

Gini index left side

$q_1 = 2r_1(1-r_1)$

Gini index right side

$q_2 = 2r_2(1-r_2)$

We define $len(\Gamma)$ to give the number of samples in the dataset, then the weighted gini value is:

$q = \frac{len(\Gamma(x_i < c))}{len(\Gamma)}*q_1 + \frac{len(\Gamma(x_i \geq c))}{len(\Gamma)}*q_2$

The goal is to find a criteria $c$ that minimizes the gini value $q$ with regards to a feature. One way to do it is to sort the data with regards to the feature, this makes $\Gamma(x < c)$ easier. We could then try every split point with a for loop and save the one that has the lowes gini value. The data is always split beteen two data points therefore the critieria c would be the average between the two data points that are sorted, i.e. $c_i = \frac{x_{i-1} + x_i}{2}$. First split has index 1. 

In [21]:
def find_split_point(data, label, feature):
    """
    data - all the data we want to split, our (gamma)
    label - the parameter we want to classify. 
    feature - the feature we want to check for, our x_i, could for example be iron
    -----------
    retrun:
    split_value - the spliting value, our c. 
    gini_value - the gini value for the best c.
    df_head - the data frame belonging to x_i < c
    df_tail - the data frame belonging to x_i => c
    """
    # Beging by sorting the data after the paramter. 
    sorted_data = data.sort_values(by=feature)
    sorted_label = sorted_data[label]
    
    # TODO loop through all the split points in the sorted data and find 
    # the best gini_value (s) and split_value (c).
    # the functions .head(split_index) and .tail(len(x) - split_index) could be useful.
    c_list = [(sorted_data[feature].iloc[i-1]+sorted_data[feature].iloc[i])/2 for i in range(1,sorted_data.shape[0])]
    q_vals = []
    for c_index in range(len(c_list)):
        split_index = c_index + 1
        r1 = np.mean(sorted_label.head(split_index))
        #print(r1)
        r2 = np.mean(sorted_label.tail(len(c_list) - split_index))

        q1 = 2*r1*(1-r1)
        q2 = 2*r2*(1-r2)

        q_vals.append(((sorted_label.head(split_index).shape[0])/(sorted_label.shape[0]))*q1 
             + ((sorted_label.tail(len(c_list) - split_index).shape[0])/(sorted_label.shape[0]))*q2)

    sorted_q = np.argsort(q_vals)
    c_index = sorted_q[0]
    split_value = c_list[c_index]
    gini_value = q_vals[c_index]
    split_value = c_list[c_index]
    split_index = c_index+1

    for i in q_vals:
        if i ==0.27655122655122655:
            print("AWIHAUWDHAOIPAWDHKLAHDNÖOAWHIDN")
   
    # TODO: get the two data frames the corresponds to the split data. 
    # the functions .head(split_index) and .tail(len(x) - split_index) could be useful. 
    df_head = sorted_data.head(split_index)
    df_tail = sorted_data.tail(len(c_list) - split_index)
 
    
    return split_value, gini_value, df_head, df_tail

In [22]:
# Test for find_split_point function, the test does not test everything and more hidden tests are used 
# during grading. 
df_test = pd.read_csv('test.csv')
split_value, gini_value, df_head, df_tail = find_split_point(df_test, 'copium', 'ground_density')

if split_value == 1.8762144694722296 and gini_value == 0.27655122655122655:
    print('Test successful')
else:
    print('Test failed')
    print("split_value = ", split_value)
    print("gini_value = ", gini_value)
    
    

Test failed
split_value =  1.8762144694722296
gini_value =  0.2714491857349


## 5 b) Tree Node

Here we will code a tree node class that can recursivly call it self. 


In [23]:
class TreeNode():
    def __init__(self, prediction=None):
        self.split_value = None # the splitting value (c)
        self.split_feature = None # what feature where uesd for the split (x_i)
        self.child_nodes = [] # list that contains two child nodes, if not leaf_node
        self.leaf_node = 0 # is this leaf_node (0=no, 1=yes)
        self.prediction = prediction # classification made in this node.
        
    def predict(self, data):
        # TODO: To make a prediction we need to traverse the tree recursivly down to a leaf node and return 
        # the prediction. 
        # step 1: check if this is a leaf node, if it is then return prediction of this node, 
        # otherwise contine with step 2.
        # step 2: Compare the input data with the splitting criteria, i.e. the is the value of the data smaller 
        # or larger then the split value.
        # step 3: Call the prediction function in the corresponding child_node and return the prediction. 
        
            
    def learn(self, data, label, min_node_size):
        """
        data - the training data
        label - the parameter we want to classify, i.e. "copium"
        min_node_size - number of data points in a node for it to become a leaf node. 
        """
        # TODO: wirte the learning function. 

        # Step 1: check if this node should be turned into a leaf node, this happens if the data set is smaller 
        # then the min_node_size or if the data labels are homogenious, i.e. avg = 0 or 1. 
        
        # Step 2: Loop over all features and get the best gini and split_value. Make sure you are not 
        # using the label as a feature. "data.columns" can give you all features plus the label 
        # Save the best split value and split_paramter and the two new data frames 
        # corresponding to the split [df_head, df_tail] (these will be data frames for the child nodes).  
        
        
        # Step 3: Calculate the majority vote prediction of the two data frames. 
        # Create two child nodes, (e.g. child = TreeNode(prediction=1) for prediction copium) and and call the 
        # learn function with the corresponding dataframe. 
        
        # Step 4: append the the child node to the self.child_nodes. It should be in the order 
        # of the child node correspoinding to [df_head, df_tail].
        
        
        

        return

IndentationError: expected an indented block after function definition on line 9 (2435152027.py, line 19)

In [ ]:
# Test to give indicaion if the TreeNode class is correct.

df_test = pd.read_csv('test.csv')
tree = TreeNode() # create root node
# learn the tree structure
tree.learn(df_test, "copium", min_node_size=4)

pred_list = []
pred_answer = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 
               1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 
               0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]

for i in range(df_test.shape[0]):
    pred = tree.predict(df_test.iloc[i])
    pred_list.append(pred)
    
if np.allclose(pred_list, pred_answer):
    print('Test successful')
else:
    print('Test failed')
    print('Prediction', pred_list)


##  Train the Tree Classifier

In [ ]:
tree = TreeNode() # create root node
# learn the tree structure
tree.learn(train, "copium", min_node_size=2)
#tree.learn(train_balanced_upsampled, "copium", min_node_size=2)


## Test Classifier

In [ ]:
y = test['copium']
x = test.drop(columns='copium')
log = Classification_eval()

for i in range(x.shape[0]):
    pred = tree.predict(x.iloc[i])
    log.update(pred, y.iloc[i])
        
print('accuarcy', log.accuracy())
print('precision', log.precision())
print('recall', log.recall())

## 5 c) part 1

Use the non balanced data to learn the tree classifier and test the classifier for some different min_node_size values.

Answer:



| min_node_size | Accuracy | Precision | Recall | 
| --- | --- | --- | --- |
| 2  |  |  |  |  
| 10 |  |  |  |   
| 20 |  |  |  |   
| 50 |  |  |  |  

## 5 c) part 2, Try with balanced data

Use the balanced data to learn the tree classifier and test the classifier for some different min_node_size values.

Answer:


| min_node_size | Accuracy | Precision | Recall | 
| --- | --- | --- | --- |
| 2  |  |  |  |  
| 10 |  |  |  |   
| 20 |  |  |  |   
| 50 |  |  |  |

## d) Tree classifier with sklearn 

Here we compare our tree classifier to the one implemented in sklearn. It is important to note that there can be differences between them, there can be minor implementation details that can affect the result, for example, there is some randomness in the sklearn implementaion. But overall they give simlar results. 

In [ ]:
# Set up data used for fitting the model
train_labels = train['copium']
train_features = train.drop(columns='copium')

# Decition tree or classifier from sklearn 
tree_sklearn = DecisionTreeClassifier(min_samples_split=2, splitter='best', 
                             criterion='gini', min_samples_leaf=1)
# Train the sklearn tree classifer
tree_sklearn.fit(train_features.values, train_labels.values)

# Train our tree classifier 
tree = TreeNode() # create root node
tree.learn(train, "copium", min_node_size=2)

# Test both and compare
y = test['copium']
x = test.drop(columns='copium')
# classify the data
for i in range(30):
    pred_sklearn = tree_sklearn.predict([x.iloc[i]])
    pred = tree.predict(x.iloc[i])

    print('sklearn pred ',pred_sklearn, 'our tree pred ', pred)

## Exersice 6: Balance data analysis

1. It there a difference between that results of the balanced and non-balanced data for the tree classifier?
2. Is there a difference between the K-NN vs tree classifier?
3. Which one would you expect to be a faster classifier?

Answer:




## Exersice 7: Deployment (Optional)

Here we will try the learned classifiers on a larger map. Make sure that the last run version of K-NN and tree have good parameters i.e. k and min_node_size values. Which one found more copium?


In [ ]:
env = Environment(map_type=2, fps=5, resolution=(1000, 1000))

try:
    sensor_properties = env.get_sensor_properties()
    sensor_sample = dict()
    for key in sensor_properties:
        sensor_sample[key] = [0]

    log_knn = Classification_eval()
    log_tree = Classification_eval()


    for i in range(500):
        action = breadth_first_search(actor=env.get_actor(), max_depth=3, action_space=env.get_action_space())
        env.step(action)
        if env.get_sensor_readings() is not None:
            sensor_readings = env.get_sensor_readings()
            for key in sensor_readings:
                sensor_sample[key][0] = sensor_readings[key]
            sensor_sample_df = pd.DataFrame(sensor_sample)
            log_knn.update(knn.predict(sensor_sample_df.iloc[0]), env.get_ground_truth())
            log_tree.update(tree.predict(sensor_sample_df.iloc[0]), env.get_ground_truth())
            env.plt_acc.update_acc(log_tree.accuracy(), log_knn.accuracy())
        env.render()

    env.exit()

    print("K-NN accuracy ", log_knn.accuracy(), "Tree accuracy", log_tree.accuracy())
    print("Number of copium deposits found, K-NN:", log_knn.TP, " Tree:", log_tree.TP)
except:
    env.exit()



